In [18]:
import pandas as pd
import numpy as np
import random
from dotenv import load_dotenv
import os
import openai
from github import Github

load_dotenv()
key = os.getenv("GITHUB_API_KEY")
g = Github(key)

### 1. Get Commits and Related Categories

In [2]:

df = pd.read_excel("./data/RQ1.xlsx")
df = df.drop(columns=["Unnamed: 2", "Count", "Count.1", "Unnamed: 5", "Unnamed: 8", "Count-subcatagory", "Count-Sub-Sub-Sub-Category", "Unnamed: 11", "Solution Category", "Tag", "Sub-Sub-Category","Sub-Sub-Sub-Category", "Sub-Sub-Sub-sub-Category"])

df

,Category,Sub-Category,Solution,Solution Main Category,Link to commit
0,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:array instead of std:Set,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/28a9cb150a...
1,NaN,NaN,NaN,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/b0379f54bd...
2,NaN,NaN,regular memory access,Data structure optimization for data locality,https://github.com/mlpack/mlpack/commit/723dea...
3,NaN,NaN,Using std:vector instead of std:List,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/8855eb54c3...
4,NaN,NaN,NaN,Avoid memory dereference by storing in register,https://github.com/lammps/lammps/commit/6c5edf...
...,...,...,...,...,...
182,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/xianyi/OpenBLAS/commit/274d...
183,NaN,NaN,NaN,fix programming error,https://github.com/libMesh/libmesh/commit/35e6...
184,NaN,NaN,NaN,fix programming error,https://github.com/mlpack/mlpack/commit/205b05...
185,Inefficiency due to new compiler version,NaN,NaN,Reduce parallelization,https://github.com/lattice/quda/commit/2ae74fd...


In [3]:
prev_category = "Inefficient coding for target micro-architecure"
prev_sub_category = "Memory/Data locality"

for row in df.iterrows():
    category = row[1]["Category"]
    if (pd.isnull(category)):
        row[1]["Category"] = prev_category
    else:
        prev_category = row[1]["Category"]
    
    sub_category = row[1]["Sub-Category"]
    if (pd.isnull(sub_category)):
        row[1]["Sub-Category"] = prev_sub_category
    else:
        prev_sub_category = row[1]["Sub-Category"]

df


,Category,Sub-Category,Solution,Solution Main Category,Link to commit
0,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:array instead of std:Set,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/28a9cb150a...
1,Inefficient coding for target micro-architecure,Memory/Data locality,NaN,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/b0379f54bd...
2,Inefficient coding for target micro-architecure,Memory/Data locality,regular memory access,Data structure optimization for data locality,https://github.com/mlpack/mlpack/commit/723dea...
3,Inefficient coding for target micro-architecure,Memory/Data locality,Using std:vector instead of std:List,Data structure optimization for data locality,https://github.com/CGAL/cgal/commit/8855eb54c3...
4,Inefficient coding for target micro-architecure,Memory/Data locality,NaN,Avoid memory dereference by storing in register,https://github.com/lammps/lammps/commit/6c5edf...
...,...,...,...,...,...
182,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/xianyi/OpenBLAS/commit/274d...
183,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/libMesh/libmesh/commit/35e6...
184,Unintentional Programming logic error,boundary condition check,NaN,fix programming error,https://github.com/mlpack/mlpack/commit/205b05...
185,Inefficiency due to new compiler version,boundary condition check,NaN,Reduce parallelization,https://github.com/lattice/quda/commit/2ae74fd...


### 2. Functions For Automatic Prompt Generation

In [19]:
# Dictionaries and globals for prompts.

category_name_to_num = {
    "Inefficient coding for target micro-architecure": 1,
    "Missing parallelism": 2,
    "Parallelization overhead/inefficiency": 3,
    "Inefficient Concurrency control and synchronization": 4,
    "Unnecessary process communiction": 5,
    "Inefficient algorithm /data-structure and their implementation": 6,
    "Inefficient memory management": 7,
    "I/O inefficiency": 8,
    "Unintentional Programming logic error": 9,
    "Inefficiency due to new compiler version ": 10
}

sub_category_name_to_num = {
    "Memory/Data locality": 1.1,
    "Micro-architectural inefficiency": 1.2,
    "Vector/SIMD parallelism": 2.1,
    "GPU parallelism": 2.2,
    "Instruction level parallelism": 2.3,
    "Task parallelism": 2.4,
    "small parallel region": 3.1,
    "Inefficeint thread mapping / inefficient block size / Load imbalance": 3.2,
    "Under-parallelization": 3.3,
    "over parallelization": 3.4,
    "Unncessary locks": 4.1,
    "Unnecessary strong memory consistency": 4.2,
    "Lock management overhead": 4.3,
    "Unnecessary synchronization": 4.4,
    "Unnecessary operation/traversal/function call": 6.1,
    "Redundant operation": 6.2,
    "Expensive operation": 6.3,
    "Frequent  function call": 6.4,
    "Inefficient data-structure library": 6.5,
    "Usage of improper data type": 6.6,
    "memory leak": 7.1,
    "repreated memory allocation": 7.2,
    "Redundant memory allocation": 7.3,
    "Slower memory allocation library call": 7.4,
    "Insufficient memory": 7.5,
    "unnecessary data copy": 7.6,
    "Sequential I/O operation": 8.1,
    "Over parallelization": 8.2
}

QUESTION = "Following code has some performance related bugs.\nPlease identify the performance bugs category and subcategory from the given list (pick only one). Moreover, please only include the category and sub-category (only number) in your response:\n\n"

LIST = "LIST:\n"

LIST += "1) Inefficient coding for target micro-architecture\n"
LIST += "   1.1) Memory/Data locality\n"
LIST += "   1.2) Micro-architectural inefficiency\n"
	
LIST += "2)	Missing parallelism\n"
LIST += "   2.1) Vector/SIMD parallelism\n"
LIST += "   2.2) GPU parallelism\n"
LIST += "   2.3) Instruction level parallelism\n"
LIST += "   2.4) Task parallelism\n"

LIST += "3)	Parallelization overhead/inefficiency\n"
LIST += "   3.1) Small parallel region\n"
LIST += "   3.2) Inefficient thread mapping / inefficient block size / Load imbalance\n"
LIST += "   3.3) Under-parallelization\n"
LIST += "   3.4) Over-Parallelization\n"

LIST += "4)	Inefficient Concurrency control and synchronization\n"
LIST += "   4.1) Unnecessary locks\n"
LIST += "   4.2) Unnecessary strong memory consistency\n"
LIST += "   4.3) Lock management overhead\n"
LIST += "   4.4) Unnecessary synchronization\n"
	
LIST += "5)	Unnecessary process communiction\n"

LIST += "6)	Inefficient algorithm /data-structure and their implementation\n"
LIST += "   6.1) Unnecessary operation/traversal/function call\n"
LIST += "   6.2) Redundant operation\n"
LIST += "   6.3) Expensive operation\n"
LIST += "   6.4) Frequent function call\n"
LIST += "   6.5) Inefficient data-structure library\n"
LIST += "   6.6) Usage of improper data type\n"
	
LIST += "7)	Inefficient memory management\n"
LIST += "   7.1) Memory leak\n"
LIST += "   7.2) Repeated memory allocation\n"
LIST += "   7.3) Redundant memory allocation\n"
LIST += "   7.4) Slower memory allocation library call\n"
LIST += "   7.5) Insufficient memory\n"
LIST += "   7.6) unnecessary data copy\n"

LIST += "8)	I/O inefficiency\n"
LIST += "   8.1) Sequential I/O operation\n"
LIST += "   8.2) Over parallelization\n"

LIST += "9) Unintentional Programming logic error\n"

LIST += "10) Inefficiency due to new compiler version\n"

In [22]:

def get_source_code(link):
    # Last 40 characters of the link is commit sha.
    sha = link[-40:]

    # Repository organization and name:
    repo = link[19:-48]

    # g is the github object created by API key.
    repo = g.get_repo(repo)
    commit = repo.get_commit(sha=sha)
    old_sha = commit.parents[0].sha
    path = commit.files[0].filename
    old_contents = repo.get_contents(path=path, ref=old_sha)
    old_code = old_contents.decoded_content.decode("utf-8")
    return old_code

  
def generate_prompt(link):
    # Automatic prompt generator from a github commit link.
    prompt = QUESTION
    prompt += LIST
    prompt += "\n"
    prompt += "CODE:\n"
    code = get_source_code(link)
    prompt += code
    prompt += "\n\n"

    # Write the prompt into a file.
    fname = "./data/prompt.txt"
    with open(fname, "w") as file:
        file.write(prompt)
    
    return prompt


# Returns (row, link, category)
def print_row(i):
    # Print the actual category and subcategory information about the input dataset.

    row = df.iloc[i]
    category = row["Category"]
    sub_category = row["Sub-Category"]
    link = row["Link to commit"]

    print(f"Row: {i}")
    print(f"Category: {category_name_to_num[category]}) {category}")
    try:
        print(f"Sub-Category: {sub_category_name_to_num[sub_category]}) {sub_category}")
    except:
        print("No subcategory")
    print(f"Link: {link}")

    # Return the found information.
    return (row, link, category)

In [17]:
# Prepare a shuffled list of bugged codes in order to ask GPT.

index_list = list(range(0, 187))
shuffled_index_list = []

while len(index_list) > 0:
    i = random.choice(index_list)
    index_list.remove(i)
    shuffled_index_list.append(i)

### 3. Classification Experiments

#### 3.1 Classification With Subcategories
* Cannot identify bugs with their subcategories.
* The results are close to a random guess.

In [ ]:
category_name_to_num = {
    "Inefficient coding for target micro-architecure": 1,
    "Missing parallelism": 2,
    "Parallelization overhead/inefficiency": 3,
    "Inefficient Concurrency control and synchronization": 4,
    "Unnecessary process communiction": 5,
    "Inefficient algorithm /data-structure and their implementation": 6,
    "Inefficient memory management": 7,
    "I/O inefficiency": 8,
    "Unintentional Programming logic error": 9,
    "Inefficiency due to new compiler version ": 10
}

sub_category_name_to_num = {
    "Memory/Data locality": 1.1,
    "Micro-architectural inefficiency": 1.2,
    "Vector/SIMD parallelism": 2.1,
    "GPU parallelism": 2.2,
    "Instruction level parallelism": 2.3,
    "Task parallelism": 2.4,
    "small parallel region": 3.1,
    "Inefficeint thread mapping / inefficient block size / Load imbalance": 3.2,
    "Under-parallelization": 3.3,
    "over parallelization": 3.4,
    "Unncessary locks": 4.1,
    "Unnecessary strong memory consistency": 4.2,
    "Lock management overhead": 4.3,
    "Unnecessary synchronization": 4.4,
    "Unnecessary operation/traversal/function call": 6.1,
    "Redundant operation": 6.2,
    "Expensive operation": 6.3,
    "Frequent  function call": 6.4,
    "Inefficient data-structure library": 6.5,
    "Usage of improper data type": 6.6,
    "memory leak": 7.1,
    "repreated memory allocation": 7.2,
    "Redundant memory allocation": 7.3,
    "Slower memory allocation library call": 7.4,
    "Insufficient memory": 7.5,
    "unnecessary data copy": 7.6,
    "Sequential I/O operation": 8.1,
    "Over parallelization": 8.2
}

QUESTION = "Following code has some performance related bugs.\nPlease identify the performance bugs category and subcategory from the given list (pick only one). Moreover, please only include the category and sub-category (only number) in your response:\n\n"

LIST = "LIST:\n"

LIST += "1) Inefficient coding for target micro-architecture\n"
LIST += "   1.1) Memory/Data locality\n"
LIST += "   1.2) Micro-architectural inefficiency\n"
	
LIST += "2)	Missing parallelism\n"
LIST += "   2.1) Vector/SIMD parallelism\n"
LIST += "   2.2) GPU parallelism\n"
LIST += "   2.3) Instruction level parallelism\n"
LIST += "   2.4) Task parallelism\n"

LIST += "3)	Parallelization overhead/inefficiency\n"
LIST += "   3.1) Small parallel region\n"
LIST += "   3.2) Inefficient thread mapping / inefficient block size / Load imbalance\n"
LIST += "   3.3) Under-parallelization\n"
LIST += "   3.4) Over-Parallelization\n"

LIST += "4)	Inefficient Concurrency control and synchronization\n"
LIST += "   4.1) Unnecessary locks\n"
LIST += "   4.2) Unnecessary strong memory consistency\n"
LIST += "   4.3) Lock management overhead\n"
LIST += "   4.4) Unnecessary synchronization\n"
	
LIST += "5)	Unnecessary process communiction\n"

LIST += "6)	Inefficient algorithm /data-structure and their implementation\n"
LIST += "   6.1) Unnecessary operation/traversal/function call\n"
LIST += "   6.2) Redundant operation\n"
LIST += "   6.3) Expensive operation\n"
LIST += "   6.4) Frequent function call\n"
LIST += "   6.5) Inefficient data-structure library\n"
LIST += "   6.6) Usage of improper data type\n"
	
LIST += "7)	Inefficient memory management\n"
LIST += "   7.1) Memory leak\n"
LIST += "   7.2) Repeated memory allocation\n"
LIST += "   7.3) Redundant memory allocation\n"
LIST += "   7.4) Slower memory allocation library call\n"
LIST += "   7.5) Insufficient memory\n"
LIST += "   7.6) unnecessary data copy\n"

LIST += "8)	I/O inefficiency\n"
LIST += "   8.1) Sequential I/O operation\n"
LIST += "   8.2) Over parallelization\n"

LIST += "9) Unintentional Programming logic error\n"

LIST += "10) Inefficiency due to new compiler version\n"

In [6]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
openai.api_key = key

In [10]:
index_list = list(range(0, 187))
shuffled_index_list = []

while len(index_list) > 0:
    i = random.choice(index_list)
    index_list.remove(i)
    shuffled_index_list.append(i)

In [27]:
IDX = 0
prompt = ""

while IDX < 187:
    current = shuffled_index_list[IDX]
    link = df.iloc[IDX]["Link to commit"]
    # print_row(current)
    print(f"-------------- IDX {IDX} --------------")
    IDX += 1
    try:
        prompt = generate_prompt(link)
    except:
        print("Error occured while generating prompt.")
        continue
    
    print("Correct categories:")
    print_row(current)
    print("-------------------------------")
    print("Reponse from GPT:")

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.2,
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )

        print()
        print(response.choices[0].message["content"])
    except:
        print("Request failed.")

-------------- IDX 0 --------------
Correct categories:
Row: 120
Category: 6) Inefficient algorithm /data-structure and their implementation
Sub-Category: 6.2) Redundant operation
Link: https://github.com/hypre-space/hypre/commit/d1fff3e84d5f6a74e601bcc89c9dcc49c6235213
-------------------------------
Reponse from GPT:
Request failed.
-------------- IDX 1 --------------
Correct categories:
Row: 89
Category: 4) Inefficient Concurrency control and synchronization
Sub-Category: 4.3) Lock management overhead
Link: https://github.com/xianyi/OpenBLAS/commit/208c7e7ca50a8bfdfabbec750bdc538023c94aed
-------------------------------
Reponse from GPT:

Category 1: Inefficient coding for target micro-architecture
Subcategory 1.2: Micro-architectural inefficiency
-------------- IDX 2 --------------
Correct categories:
Row: 140
Category: 6) Inefficient algorithm /data-structure and their implementation
Sub-Category: 6.3) Expensive operation
Link: https://github.com/OpenFOAM/OpenFOAM-dev/commit/56059

#### 3.2 Classification Without Subcategories

In [6]:
# Connect to OpenAI API.

load_dotenv()
key = os.getenv("OPENAI_API_KEY")
openai.api_key = key

In [20]:
# Necessary globals for this experiment.

category_name_to_num = {
    "Inefficient coding for target micro-architecure": 1,
    "Missing parallelism": 2,
    "Parallelization overhead/inefficiency": 3,
    "Inefficient Concurrency control and synchronization": 4,
    "Unnecessary process communiction": 5,
    "Inefficient algorithm /data-structure and their implementation": 6,
    "Inefficient memory management": 7,
    "I/O inefficiency": 8,
    "Unintentional Programming logic error": 9,
    "Inefficiency due to new compiler version ": 10
}

In [21]:
# Necessary globals for this experiment.

QUESTION = "Following code has some performance related bugs.\nPlease identify the performance bugs category from the given list and explain it.\n\n"

LIST = "LIST:\n"
LIST += "1) Inefficient coding for target micro-architecture\n"
LIST += "2)	Missing parallelism\n"
LIST += "3)	Parallelization overhead/inefficiency\n"
LIST += "4)	Inefficient Concurrency control and synchronization\n"
LIST += "5)	Unnecessary process communiction\n"
LIST += "6)	Inefficient algorithm /data-structure and their implementation\n"
LIST += "7)	Inefficient memory management\n"
LIST += "8)	I/O inefficiency\n"
LIST += "9) Unintentional Programming logic error\n"
LIST += "10) Inefficiency due to new compiler version\n"

In [24]:
# Prepare the codes which are not too lengthy to ask to GPT.

experiment_index_list = []

IDX = 0
prompt = ""

while IDX < 187:
    current = shuffled_index_list[IDX]
    link = df.iloc[IDX]["Link to commit"]
    IDX += 1
    try:
        prompt = generate_prompt(link)
    except:
        continue
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.2,
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )
        experiment_index_list.append(IDX)
    except:
        pass

In [26]:
experiment_list = []

for i in experiment_index_list:
    experiment_list.append(shuffled_index_list[i])

experiment_list

[167,
 116,
 149,
 18,
 146,
 75,
 5,
 78,
 147,
 40,
 134,
 16,
 160,
 45,
 128,
 156,
 55,
 58,
 47,
 133,
 35,
 139,
 15,
 91,
 186,
 64,
 166,
 81,
 104,
 174,
 129,
 83,
 85,
 90,
 155,
 130,
 141,
 169,
 13,
 137,
 110,
 27,
 43,
 38,
 184,
 164,
 100,
 61,
 93,
 183,
 6,
 62,
 157,
 150,
 117,
 103,
 17,
 24,
 185,
 0,
 131,
 158,
 126,
 39,
 96,
 175,
 25,
 179,
 162,
 23,
 71,
 98,
 1,
 42,
 79,
 114,
 41]

In [29]:
# Experiment: Ask ChatGPT to classify the bugs.
# Pass a code if the prompt becomes too lengthy, which makes it impossible to give to GPT.
# Output the responses stdout to later save them.
results_df = pd.DataFrame(columns=["idx", "link", "bug_category", "response"])

IDX = 0
prompt = ""

while IDX < len(experiment_list):
    current = experiment_list[IDX]
    link = df.iloc[IDX]["Link to commit"]
    IDX += 1
    try:
        prompt = generate_prompt(link)
    except:
        results_df.loc[current] = ["-", "-", "-", "-"]
        continue
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.2,
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )
        print(f"-------------- IDX {IDX} --------------")
        print("Correct categories:")
        row, link, category = print_row(current)
        print("-------------------------------")
        print("Reponse from GPT:")
        print(response.choices[0].message["content"])
        results_df.loc[current] = [IDX, link, category, response.choices[0].message["content"]]
    except:
        results_df.loc[current] = ["-", "-", "-", "-"]
        pass
        # print("Request failed.")

-------------- IDX 2 --------------
Correct categories:
Row: 116
Category: 6) Inefficient algorithm /data-structure and their implementation
Sub-Category: 6.2) Redundant operation
Link: https://github.com/mlpack/mlpack/commit/5e1d0dacd0121243ba1aa9258992931d0839666d
-------------------------------
Reponse from GPT:
There are no obvious performance related bugs in the given code.
-------------- IDX 3 --------------
Correct categories:
Row: 149
Category: 6) Inefficient algorithm /data-structure and their implementation
Sub-Category: 6.3) Expensive operation
Link: https://github.com/mlpack/mlpack/commit/83207c4439ce3072e9b590cd6d2feeafdd25f2bd
-------------------------------
Reponse from GPT:
The performance bug category in the given code is 6) Inefficient algorithm/data-structure and their implementation. 

Explanation: 
1. The code is using a priority queue to store the computation frontier, which can be inefficient for large data sets as it has a high overhead for insertion and deletio

In [31]:
results_df.to_excel("./data/experiment_results/experiment_3.xlsx")

### 4. Fixing Bugs in Parallel Code

In [94]:
# Connect to OpenAI API.

load_dotenv()
key = os.getenv("OPENAI_API_KEY")
openai.api_key = key

#### 4.1 Save Bugged Codes From Github

In [68]:
# Get bugged code files from GitHub

# Return source code and filename
def get_source_code(link):
    # Last 40 characters of the link is commit sha.
    sha = link[-40:]

    # Repository organization and name:
    repo = link[19:-48]

    # g is the github object created by API key.
    repo = g.get_repo(repo)
    commit = repo.get_commit(sha=sha)
    old_sha = commit.parents[0].sha
    path = commit.files[0].filename
    old_contents = repo.get_contents(path=path, ref=old_sha)
    old_code = old_contents.decoded_content.decode("utf-8")

    idx = -1
    try:
        idx = path.rfind("/")
    except:
        pass
    
    fname = path[idx + 1:]
    return old_code, fname

# Export a code into file
def create_file(code, fname):
    f = open(fname, "w")
    f.write(code)
    f.close()

In [101]:
# Download file content in RQ1 from GitHub.

fpath = "./data/bugged_codes/"
links = df["Link to commit"].copy()

for i, link in enumerate(links):
    try:
        code, fname = get_source_code(link)
        fname = str(i + 2) + "_" + fname
        create_file(code, fpath + fname)
    except:
        print(f"Error while downloading file: {link}")

    

Error while downloading file: https://github.com/openmm/openmm/commit/b1be68d873e649ab07ae6c65f017163066373123
Error while downloading file: https://github.com/lattice/quda/commit/e04b189e232392b36d0140bdb53bd5d0af9d05b6
Error while downloading file: nan


#### 4.2 Request Fixes From ChatGPT

In [85]:
def request_code_fix(prompt):
    # Request a code fix 
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0.2,
        max_tokens=1000,
        messages=[{"role": "user", "content": prompt}]
    )

    # Process the response to include the code block
    idx = -1
    idx2= -1
    code = ""
    explanation = ""
    msg = response.choices[0].message.content
    try:
        msg = response.choices[0].message.content
        idx = msg.find("```")
        idx2 = msg.find("```", idx+1)
        code = msg[idx: idx2]
        code = code[4:-3]
        explanation = msg[:idx]
        if (idx2 != -1 and idx != -1):
            explanation += "\n<CODE>\n"
            explanation += msg[idx2+3:]
    except:
        explanation = msg

    return code, explanation

In [100]:
question = "The following code has some performance bugs. Please find and fix the bug. Also, please include one code block containing the full fixed code in your response\n"

target_path_codes = "./data/fixed_codes/"
target_path_explanations = "./data/bugfix_explanations/"
 
# Iterate over bugged codes:
directory = "./data/bugged_codes/"

for filename in os.listdir(directory):
    idx = filename[:filename.find("_")]
    rest = filename[filename.find("_")+1:]

    # Get content of the file
    f = open(directory + filename, "r")
    code = f.read()
    f.close()

    try:
        # Get a fix from ChatGPT and save it to bugfix_explanations and fixed_codes.
        fixed_code, explanation = request_code_fix(question + "\n" + code)
        create_file(fixed_code, target_path_codes + str(int(idx) + 2) + "_fixed_" + rest)
        create_file(explanation, target_path_explanations + str(int(idx) + 2) + "_explanation.txt")
    except:
        print(f"An error occured in request: {filename}")


An error occured in request: 123_ns.c
An error occured in request: 12_sgemm_small_kernel_nn_skylakex.c
An error occured in request: 138_nb_free_energy.c
An error occured in request: 156_neighbor_search_impl.hpp
An error occured in request: 116_diagonal_gmm_impl.hpp
An error occured in request: 48_param.h
An error occured in request: 145_sparsity_internal.cpp
An error occured in request: 6_sbgemm_kernel_16x4_cooperlake.c
An error occured in request: 91_nbnxn_cuda_kernel.cuh
An error occured in request: 178_matmul.c
An error occured in request: 35_restrictor.cu
An error occured in request: 53_zdot_microk_haswell-2.c
An error occured in request: 86_AABB_tree.h
An error occured in request: 181_disk.arrays.c
An error occured in request: 67_HISTORY.md
An error occured in request: 51_param.h
An error occured in request: 113_gemm.c
An error occured in request: 25_convolve.hpp
An error occured in request: 72_bilininteg_diffusion_pa.cpp
An error occured in request: 93_parallel_ghost_sync.h
An er